In [ ]:
# Treinamento do modelo LSTM com dados do banco

# 1. Conceito básico sobre LSTM
"""
LSTM (Long Short-Term Memory) é uma variante das redes neurais recorrentes (RNNs) projetada para aprender dependências de longo prazo em séries temporais.
Ela é ideal para prever sequências de dados com padrões temporais, como vendas mensais, sinais de sensores, ou uso de medicamentos ao longo do tempo.
"""

In [ ]:

# 2. Verificar uso de GPU
import tensorflow as tf
print("GPU disponível:", tf.config.list_physical_devices('GPU'))

In [ ]:
print("TensorFlow está usando GPU?", tf.test.is_built_with_cuda())
print("Dispositivos visíveis:", tf.config.list_physical_devices())

In [12]:

# 3. Carregar dados do banco de dados
import os
import pandas as pd
from sqlalchemy import create_engine

db_path = os.path.abspath("../storage/anvisa.db")

# Conecta ao banco SQLite
engine = create_engine(f"sqlite:///{db_path}")

# Exemplo de query (filtrar por substância ou UF para simplificar a série)
query = """
SELECT ano_venda, mes_venda, qtd_unidade_farmacotecnica
FROM vendas_manipulados
WHERE tipo_unidade_farmacotecnica = 'CÁPSULA'  AND principio_ativo = 'CLORIDRATO DE FLUOXETINA' AND uf_venda = 'SP'
"""

# Executa consulta
df = pd.read_sql(query, engine)

In [ ]:
import os
# Caminho relativo com base no notebook que está na pasta 'notebooks/'
db_path = os.path.abspath("../storage/anvisa.db")
print("Caminho correto:", db_path)

In [ ]:

# 4. Processar datas e normalizar
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Cria campo de data
df['DATA'] = pd.to_datetime(df['ano_venda'].astype(str) + '-' + df['mes_venda'].astype(str))
df.set_index('DATA', inplace=True)

# Agrupar por mês (caso haja dados duplicados)
df = df[['qtd_unidade_farmacotecnica']].resample('M').sum()
df = df.dropna()

# Normaliza
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

In [ ]:

# 5. Criar sequências para LSTM
def criar_sequencias(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 3
X, y = criar_sequencias(df_scaled, seq_length)

# Divide em treino e teste
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [ ]:
# 6. Construir e treinar modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Treinamento
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

In [ ]:
# 7. Avaliar e plotar previsão
import matplotlib.pyplot as plt

# Prever
y_pred = model.predict(X_test)

# Desnormalizar
y_pred = scaler.inverse_transform(y_pred)
y_test_real = scaler.inverse_transform(y_test)

# Plot
plt.figure(figsize=(10,5))
plt.plot(y_test_real, label="Real")
plt.plot(y_pred, label="Previsto")
plt.legend()
plt.title("Previsão com LSTM")
plt.xlabel("Tempo")
plt.ylabel("Qtd Unidades")
plt.grid(True)
plt.show()

In [ ]:
# 8. Salvar modelo treinado
model.save("artifacts/modelo_lstm.keras")